# Track notebooks & scripts

This guide explains how to use {meth}`~lamindb.track` & {meth}`~lamindb.finish` to track notebook & scripts along with their inputs and outputs.

For tracking data lineage in pipelines, see {doc}`docs:pipelines`.

In [ ]:
# !pip install 'lamindb[jupyter]'
!lamin init --storage ./test-track

## Track data lineage

Call {meth}`~lamindb.track` to register a data transformation and start tracking inputs & outputs of a run. You will find your notebook or script in the {class}`~lamindb.Transform` registry along with pipelines & functions. {class}`~lamindb.Run` stores executions.

In [ ]:
import lamindb as ln

# --> `ln.track()` generates a uid for your code
# --> `ln.track(uid)` initiates a tracked run
ln.track("9priar0hoE5u0000")

# your code

ln.finish()  # mark run as finished, save execution report, source code & environment

LaminDB now automatically tracks all input and output data of the current run.

Note: You can access a `requirements.txt` via `ln.context.run.environment`.

Below is how [a notebook](https://lamin.ai/laminlabs/lamindata/transform/PtTXoc0RbOIq65cN) with run report looks on the hub.

<img src="https://lamin-site-assets.s3.amazonaws.com/.lamindb/RGXj5wcAf7EAc6J8dJfH.png" width="900px">

## Query & load a notebook or script

In the API, filter {class}`~lamindb.Transform` to obtain a transform record:

```python
transform = ln.Transform.get(name="Track notebooks & scripts")
transform.source_code  # source code
transform.latest_run.report  # report of latest run
transform.latest_run.environment  # environment of latest run
transform.runs  # all runs
```

On the hub, search or filter the `transform` page and then load a script or notebook on the CLI. For example,

```bash
lamin load https://lamin.ai/laminlabs/lamindata/transform/13VINnFk89PE0004
```

## Sync scripts with GitHub

To sync with your git commit, add the following line to your script:

```python
ln.settings.sync_git_repo = <YOUR-GIT-REPO-URL>
```

```{eval-rst}
.. literalinclude:: scripts/synced-with-git.py
   :language: python
   :caption: synced-with-git.py
```

You'll now see the GitHub emoji clickable on the hub.

<img src="https://lamin-site-assets.s3.amazonaws.com/.lamindb/IpV8Kiq4xUbgXhzlUYT7.png" width="900px">
<br>

## Track run parameters

LaminDB's validation dialogue auto-generates code for run parameters. Here is an example:

In [ ]:
ln.Param(name="input_dir", dtype="str").save()
ln.Param(name="learning_rate", dtype="float").save()
ln.Param(name="downsample", dtype="bool").save()

```{eval-rst}
.. literalinclude:: scripts/run-track-with-params.py
   :language: python
   :caption: run-track-with-params.py
```

Run the script.

In [ ]:
!python scripts/run-track-with-params.py  --input-dir ./mydataset --learning-rate 0.01 --downsample

## Query by run parameters

Query for all runs that match a certain parameters:

In [ ]:
ln.Run.params.filter(learning_rate=0.01, input_dir="./mydataset").df()

Look at the parameter values that were used for a given run.

In [ ]:
run = ln.Run.params.filter(learning_rate=0.01).order_by("-started_at").first()
run.params.get_values()

## Manage notebook templates

A notebook acts like a template upon using `lamin load` to load it. Consider you run:

```bash
lamin load https://lamin.ai/account/instance/transform/Akd7gx7Y9oVO0000
```

Upon running the returned notebook, you'll automatically create a new version and be able to browse it via the version dropdown on the UI.

Additionally, you can:

- label using `ULabel`, e.g., `transform.ulabels.add(template_label)`
- tag with an indicative `version` string, e.g., `transform.version = "T1"; transform.save()`

:::{dropdown} Saving a notebook as an artifact

Sometimes you might want to save a notebook as an artifact. This is how you can do it:

```bash
lamin save template1.ipynb --key templates/template1.ipynb --description "Template for analysis type 1" --registry artifact
```

:::

In [ ]:
assert run.params.get_values() == {'downsample': True, 'input_dir': './mydataset', 'learning_rate': 0.01}

# clean up test instance
!rm -r ./test-track
!lamin delete --force test-track